# Block 2: Essential Data Wrangling with NumPy & Pandas

**Python Module for Incoming ISE & OR PhD Students**  
Instructor: Will Kirschenman | August 7, 2025 | 10:00 AM - 10:50 AM

---

## Welcome to Block 2! 🐺

In Block 1, we learned the programming mindset and Python basics. Now we're diving into the real power of Python for data science: **NumPy** and **Pandas**. These are the tools that make Python the go-to language for data analysis in research.

By the end of this block, you'll be able to:
- Work with NumPy arrays for efficient numerical computations
- Load, explore, and manipulate data with Pandas DataFrames
- Clean messy real-world data (the kind you'll encounter in research)
- Prepare a clean dataset for machine learning (Block 3!)

**Our Mission**: We have a dataset of NC State PhD student research productivity that's... well, let's just say it needs some TLC. We'll learn NumPy and Pandas while turning this messy data into something beautiful and analysis-ready.

In [ ]:
# 📦 Package Installation & Setup
# Run this cell ONLY if you encounter import errors
# Most packages are pre-installed in Google Colab

import sys
import subprocess

def install_package(package_name):
    """Install a package using pip if not already installed"""
    try:
        __import__(package_name)
        print(f"✅ {package_name} already installed")
    except ImportError:
        print(f"📦 Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"✅ {package_name} installed successfully")

# Core packages used in this notebook (Block 2: Data Wrangling)
required_packages = [
    'numpy',
    'pandas', 
    'matplotlib'
]

print("🔍 Checking required packages for Block 2...")
print("=" * 45)

for package in required_packages:
    install_package(package)

print("\n🎉 All packages ready! You can now run all cells without import errors.")
print("💡 Tip: In Google Colab, most packages are pre-installed, so you likely won't need to install anything!")

## Setup: Import Our Tools

Let's start by importing the libraries we'll need. In Google Colab, these are already installed!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Creating Our Dataset

Since we're working in Google Colab, let's create our synthetic dataset right here. This represents research productivity data for PhD students at NC State - but it's messy, just like real data!

In [ ]:
np.random.seed(2025)
random.seed(2025)

# Generate synthetic PhD student data
def create_phd_dataset():
    """
    Create a realistic but messy dataset of PhD student research productivity
    """
    # For reproducible results, uncomment the following two lines
    # np.random.seed(42)
    # random.seed(42)
    
    departments = {
        'ISE': ['ISE', 'I.S.E.', 'Industrial Systems', 'Industrial & Systems Engineering'],
        'OR': ['OR', 'O.R.', 'Operations Research', 'Operations Res.'],
        'CSC': ['CSC', 'Computer Science', 'CS', 'Comp Sci'],
        'ECE': ['ECE', 'Electrical & Computer Engineering', 'E.C.E.', 'EE'],
        'MAE': ['MAE', 'Mechanical & Aerospace Engineering', 'M.A.E.', 'Mechanical']
    }
    
    print("Type of departments:", type(departments))  # Ensure departments is a dictionary
    
    n_students = 280
    data = []
    
    for i in range(n_students):
        # Basic info
        student_id = f"PhD_{i+1:03d}"
        
        # Department (with inconsistent formatting)
        dept_clean = random.choice(list(departments.keys()))
        dept_messy = random.choice(departments[dept_clean])
        
        # Years in program (1-7, with most being 2-4)
        years = np.random.choice([1, 2, 3, 4, 5, 6, 7], p=[0.1, 0.2, 0.25, 0.2, 0.15, 0.05, 0.05])
        
        # Papers published (correlated with years, but with variation)
        base_papers = max(0, years - 1 + np.random.normal(0, 1.5))
        papers = max(0, int(base_papers))
        
        # Conferences attended (somewhat correlated with papers)
        conferences = max(0, int(papers * 0.8 + np.random.normal(0, 1)))
        
        # Coffee consumption (PhD students love coffee, with some outliers)
        coffee_base = np.random.normal(3.5, 1.5)
        coffee = max(0, coffee_base)
        
        # Hours in Hunt Library (varies widely, some night owls)
        hunt_hours = max(0, np.random.normal(25, 12))
        
        # Advisor meetings per month (1-8, mostly 2-4)
        advisor_meetings = max(1, int(np.random.normal(3, 1.5)))
        
        # Stress level (1-10, correlated with years and workload)
        stress_base = 3 + years * 0.5 + papers * 0.3 + np.random.normal(0, 1.5)
        stress = max(1, min(10, stress_base))
        
        # Funding amount (varies by department and year)
        funding_base = 25000 + years * 1000 + np.random.normal(0, 3000)
        funding = max(15000, funding_base)
        
        # Distance from campus (most live close, some commute)
        distance = max(0.1, np.random.exponential(3))
        
        # Add some missing values
        if random.random() < 0.08:  # 8% missing coffee data
            coffee = np.nan
        if random.random() < 0.05:  # 5% missing stress data
            stress = np.nan
        if random.random() < 0.03:  # 3% missing funding data
            funding = np.nan
        
        data.append({
            'student_id': student_id,
            'department': dept_messy,
            'years_in_program': years,
            'papers_published': papers,
            'conferences_attended': conferences,
            'coffee_cups_per_day': coffee,
            'hours_in_hunt_library_per_week': hunt_hours,
            'advisor_meetings_per_month': advisor_meetings,
            'stress_level': stress,
            'funding_amount': funding,
            'distance_from_campus_miles': distance
        })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Add some duplicate rows (for cleaning exercise)
    duplicates = df.sample(n=8, random_state=42)
    df = pd.concat([df, duplicates], ignore_index=True)
    
    # Add some extreme outliers for coffee consumption
    outlier_indices = np.random.choice(df.index, size=5, replace=False)
    df.loc[outlier_indices, 'coffee_cups_per_day'] = np.random.uniform(15, 25, size=5)
    
    return df

# Create our dataset
df_raw = create_phd_dataset()

print(f"🎯 Dataset created with {len(df_raw)} rows")
print(f"📊 Columns: {list(df_raw.columns)}")
print(f"❌ Missing values: {df_raw.isnull().sum().sum()}")
print(f"🔄 Duplicate rows: {df_raw.duplicated().sum()}")
print("\n🔍 Let's peek at the first few rows...")
df_raw.head()

## 😬 Houston, We Have Problems...

Look at that data! It's messy:
- Missing values (NaN)
- Duplicate rows
- Inconsistent department names
- Probably some outliers too

**Don't panic!** This is exactly the kind of data you'll encounter in real research. By the end of this block, we'll have it sparkling clean and ready for analysis.

But first, let's learn the tools we need...

---

# Part 1: NumPy Fundamentals 🔢

## Why NumPy?

NumPy (Numerical Python) is the foundation of the Python data science ecosystem. It provides:
- **Speed**: Operations are implemented in C, making them much faster than pure Python
- **Memory efficiency**: Arrays use less memory than Python lists
- **Vectorization**: Perform operations on entire arrays at once

Let's see the speed difference:

In [ ]:
import time

# Create large datasets
n_elements = 1000000
python_list = list(range(n_elements))
numpy_array = np.array(python_list)

# Time Python list operation
start_time = time.perf_counter()
python_result = [x * 2 for x in python_list]
python_time = time.perf_counter() - start_time

# Time NumPy operation
start_time = time.perf_counter()
numpy_result = numpy_array * 2
numpy_time = time.perf_counter() - start_time

print("--- Performance ---")
print(f"🐌 Python list: {python_time:.4f} seconds")
print(f"🚀 NumPy array: {numpy_time:.4f} seconds")
print(f"⚡ NumPy is {python_time/numpy_time:.1f}x faster!")

# --- Memory Usage Comparison ---

# Python list memory calculation (more accurately, the memory of the list object itself
# plus the memory of the integer objects it points to)
# This is a good approximation without iterating over every element.
# On a 64-bit system, a standard integer object is 28 bytes.
# We also have to account for the list object itself and the pointers.
# A simpler, more direct approach is to show the difference
# in how memory is stored.
python_list_mem_approx = sys.getsizeof(python_list) + n_elements * sys.getsizeof(python_list[0])
numpy_array_mem = numpy_array.nbytes

print("\n--- Memory Usage ---")
print(f"🧠 Python list memory: {python_list_mem_approx / 1024 / 1024:.2f} MB (approx.)")
print(f"🧠 NumPy array memory: {numpy_array_mem / 1024 / 1024:.2f} MB")
print(f"💡 NumPy uses {python_list_mem_approx / numpy_array_mem:.1f}x less memory!")

# --- Conclusion ---
print("\nThis is why we use NumPy for numerical computing! It's both faster and more memory-efficient.")

## Creating NumPy Arrays

There are several ways to create NumPy arrays:

In [ ]:
# From a Python list
coffee_consumption = np.array([3.2, 4.1, 2.8, 5.0, 3.7])
print(f"☕ Coffee consumption: {coffee_consumption}")
print(f"Type: {type(coffee_consumption)}") # numpy n-dimensional array object

# Create arrays with specific values
zeros = np.zeros(5)  # Array of zeros
ones = np.ones(3)    # Array of ones
range_array = np.arange(1, 8)  # Range from 1 to 7 (like range() but NumPy)

print(f"\n🔢 Zeros: {zeros}")
print(f"🔢 Ones: {ones}")
print(f"🔢 Range (PhD years): {range_array}")

# Random arrays (useful for simulations)
random_stress = np.random.normal(5, 2, 10)  # Mean=5, std=2, 10 samples
print(f"\n😰 Random stress levels: {random_stress}")

In [ ]:
# TODO: Create the following arrays using NumPy functions:

# 1. Create an array of 10 zeros to represent baseline measurements
# Hint: Use a function that creates an array of a specified size and fills it with a single value.
baseline_measurements = ???

# 2. Create an array of survey scores from 1 to 5 (inclusive)
# Hint: Use a function similar to Python's `range()` but for NumPy arrays.
survey_scale = ???

# 3. Create an array of 8 ones to represent control group results
# Hint: This is similar to creating an array of zeros, but with a different fill value.
control_group = ???

# 4. Generate 15 random experiment results between 0 and 100
# Hint: try `np.random.randint()` or `np.random.uniform()`, similar to the random stress example above.
experiment_results = ???

# 5. Create an array with values [2, 4, 6, 8, 10] using arange
# Hint: The `arange` function can take an optional step size.
even_numbers = ???

print("✅ Array Creation Exercise:")
print(f"Baseline measurements: {baseline_measurements}")
print(f"Survey scale: {survey_scale}")
print(f"Control group: {control_group}")
print(f"Experiment results: {experiment_results}")
print(f"Even numbers: {even_numbers}")

# Check your work:
print(f"\n🔍 Verification:")
print(f"Baseline has {len(baseline_measurements)} zeros: {np.all(baseline_measurements == 0)}")
print(f"Survey scale goes 1-5: {np.array_equal(survey_scale, [1, 2, 3, 4, 5])}")
print(f"Control group has {len(control_group)} ones: {np.all(control_group == 1)}")
print(f"Experiment results shape: {experiment_results.shape}")
print(f"Even numbers are correct: {np.array_equal(even_numbers, [2, 4, 6, 8, 10])}")

## Basic NumPy Operations

NumPy shines with vectorized operations - you can perform calculations on entire arrays:

In [ ]:
# Let's work with some PhD student data
papers_per_year = np.array([0, 1, 2, 1, 3, 2, 1])  # Papers published each year
conference_costs = np.array([1200, 1500, 800, 2000, 1100])  # Conference registration costs

# Mathematical operations
total_papers = np.sum(papers_per_year)
avg_papers = np.mean(papers_per_year)
max_papers = np.max(papers_per_year)

print(f"📄 Total papers published: {total_papers}")
print(f"📊 Average papers per year: {avg_papers:.2f}")
print(f"🏆 Best year (max papers): {max_papers}")

# Vectorized operations
papers_doubled = papers_per_year * 2  # What if we were twice as productive?
print(f"\n🚀 If we doubled our productivity: {papers_doubled}")

# Statistical operations
print(f"\n💰 Conference costs statistics:")
print(f"   Mean: ${np.mean(conference_costs):.2f}")
print(f"   Median: ${np.median(conference_costs):.2f}")
print(f"   Std Dev: ${np.std(conference_costs):.2f}")

## 🎯 Mini Exercise: Research Metrics Analysis

Now that you've learned the statistical functions, practice using them with research data:

In [ ]:
# Simulated research data
experiment_scores = np.array([85, 92, 78, 95, 88, 91, 79, 86, 93, 84, 77, 89])
control_scores = np.array([82, 87, 75, 90, 85, 88, 76, 83, 89, 81])

# TODO: Calculate the following statistics:

# 1. Mean score for each group
experiment_mean = ???
control_mean = ???

# 2. Standard deviation for each group  
experiment_std = ???
control_std = ???

# 3. The difference between group means
mean_difference = ???

# 4. Find the highest and lowest scores in the experiment group
experiment_max = ???
experiment_min = ???

# 5. Count how many experiment scores are above 85
# Hint: A comparison like `experiment_scores > 85` will give you a boolean array.
# What is a fast way to count the `True` values in a NumPy array?
above_85_count = ???

print("📊 Research Analysis Results:")
print(f"Experiment group mean: {experiment_mean:.2f}")
print(f"Control group mean: {control_mean:.2f}")
print(f"Difference in means: {mean_difference:.2f}")
print(f"Experiment std dev: {experiment_std:.2f}")
print(f"Control std dev: {control_std:.2f}")
print(f"Experiment range: {experiment_min} to {experiment_max}")
print(f"Experiment scores above 85: {above_85_count} out of {len(experiment_scores)}")

# Bonus: Which group performed better?
if experiment_mean > control_mean:
    print("🏆 The experiment group performed better!")
else:
    print("🏆 The control group performed better!")

## 🎯 Mini Exercise: Hunt Library Hours

Analyze the Hunt Library hours data:

In [ ]:
# Extract Hunt Library hours data (removing NaN values)
hunt_hours = df_raw['hours_in_hunt_library_per_week'].dropna().values

# TODO: Calculate and print these statistics:
# 1. Mean hours per week
# 2. How many students spend more than 40 hours/week in Hunt Library
# 3. What percentage of students basically live in Hunt Library (>50 hours/week)?

# Your code here:
mean_hours = ???
over_40_hours = ???
over_50_hours = ???
percentage_over_50 = ???

print(f"📚 Hunt Library analysis for {len(hunt_hours)} students:")
print(f"   Mean hours per week: {mean_hours:.2f}")
print(f"   Students spending >40 hours/week: {over_40_hours}")
print(f"   Students basically living in Hunt Library (>50 hours/week): {over_50_hours}")
print(f"   Percentage living in Hunt Library: {percentage_over_50:.1f}%")

---

# Part 2: Pandas - Your Data Analysis Best Friend 🐼

## NumPy vs Pandas: When to Use What?

- **NumPy**: Best for numerical computations, homogeneous data, mathematical operations
- **Pandas**: Best for mixed data types, labeled data, data cleaning, real-world datasets

Pandas is built on top of NumPy, so you get the speed benefits plus much more functionality!

## Meet the DataFrame

The DataFrame is like a supercharged spreadsheet. Let's explore our PhD student data:

In [ ]:
# Let's start fresh with our raw data
df = df_raw.copy()  # Make a copy so we don't mess up the original

# df.shape shows the number of rows and columns; df.columns lists the column names
print("📊 DataFrame Shape:", df.shape)
print("\n📋 Column Names:")
for i, col in enumerate(df.columns):
    print(f"   {i}: {col}")

print("\n🔍 Data Types:")
print(df.dtypes) # df.dtypes shows the data type of each column

## Essential DataFrame Methods

These are the methods you'll use constantly:

.head() shows the first few rows

In [ ]:
# First few rows
print("🔝 First 5 rows:")
print(df.head())

.tail() shows the last few rows

In [ ]:
# Last few rows
print("\n🔚 Last 3 rows:")
print(df.tail(3))

.info() gives a summary of the DataFrame including non-null counts

In [ ]:
# General information about the dataset
print("ℹ️  Dataset Info:")
print(df.info())

.describe() gives summary statistics for numerical columns

In [ ]:
# Statistical summary
print("📈 Statistical Summary:")
print(df.describe())

## Data Selection and Indexing

Pandas offers multiple ways to select data:

In [ ]:
# Select a single column
departments = df['department']
print(f"🏢 Department column type: {type(departments)}")
print(f"🏢 Unique departments: {departments.unique()}")
print(f"🏢 Department counts:")
print(departments.value_counts())

In [ ]:
# Select multiple columns
student_basics = df[['student_id', 'department', 'years_in_program']]
print("👥 Student basics:")
print(student_basics.head())

In [ ]:
# Filter rows based on conditions
# Let's find the seasoned PhD students (4+ years)
veterans = df[df['years_in_program'] >= 4]
print(f"🎓 Veterans (4+ years): {len(veterans)} students")
print(f"📊 Their average papers: {veterans['papers_published'].mean():.2f}")

# Multiple conditions
ise_veterans = df[(df['years_in_program'] >= 4) & (df['department'].str.contains('ISE', na=False))]
print(f"\n🏭 ISE veterans: {len(ise_veterans)} students")
print(f"📊 Their average papers: {ise_veterans['papers_published'].mean():.2f}")

## 🎯 Mini Exercise: Department Analysis

Let's analyze our departments, but first we need to deal with those inconsistent names:

In [ ]:
# Look at the department name mess
print("🏢 Department name variations:")
print(df['department'].value_counts())

# Let's clean this up as we learn!
def standardize_department(dept_name):
    """Clean up department names"""
    if pd.isna(dept_name):
        return dept_name # Keep NaN as is
    
    dept_name = str(dept_name).strip() # Remove leading/trailing whitespace
    
    if 'Industrial' in dept_name or dept_name in ['ISE', 'I.S.E.']:
        return 'ISE'
    elif 'Operations' in dept_name or dept_name in ['OR', 'O.R.']:
        return 'OR'
    elif 'Computer' in dept_name or dept_name in ['CSC', 'CS', 'Comp Sci']:
        return 'CSC'
    elif 'Electrical' in dept_name or dept_name in ['ECE', 'E.C.E.', 'EE']:
        return 'ECE'
    elif 'Mechanical' in dept_name or dept_name in ['MAE', 'M.A.E.']:
        return 'MAE'
    else:
        return dept_name

# Apply the cleaning function
df['department_clean'] = df['department'].apply(standardize_department)

print("\n✨ Cleaned department names:")
print(df['department_clean'].value_counts())

### 🤔 Your Turn: Analyze by Department

Now use the cleaned department data to answer these questions:

In [ ]:
# TODO: Find the answers to these questions:
# 1. Which department has the highest average papers published?
# 2. Which department spends the most time in Hunt Library on average?
# 3. Which department has the highest stress levels?

# Your code here:
print("📊 Department Analysis:")

print("\n📄 Average papers by department:")
papers_by_dept = df.groupby('department_clean')['papers_published'].mean().sort_values(ascending=False)
print(papers_by_dept)

print("\n📚 Average Hunt Library hours by department:")
# Hint: This is very similar to the 'papers_by_dept' calculation.
# Group by 'department_clean', select the 'hours_in_hunt_library_per_week' column,
# calculate the mean, and then sort in descending order.
hunt_by_dept = ???
print(hunt_by_dept)

print("\n😰 Average stress level by department:")
# Hint: Group by 'department_clean' and calculate the mean of 'stress_level'.
# This time, sort the results in ASCENDING order (True) to see the lowest stress levels first.
# To find the HIGHEST stress levels, you'll need to look at the end of the sorted list!
stress_by_dept = ???
print(stress_by_dept)

---

# Part 3: Data Cleaning Workshop 🧹

Now for the main event! Let's clean our messy dataset step by step.

## Step 1: Handle Missing Values

Let's see what we're dealing with:

In [ ]:
# Check for missing values
print("❌ Missing values by column:")
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0]
print(missing_summary)

# Percentage of missing values
print("\n📊 Missing value percentages:")
missing_percentages = (df.isnull().sum() / len(df)) * 100
missing_percentages = missing_percentages[missing_percentages > 0]
print(missing_percentages)

### Different Strategies for Missing Values:

1. **Remove rows** with missing values (if very few)
2. **Fill with mean/median** (for numerical data)
3. **Fill with mode** (for categorical data)
4. **Forward/backward fill** (for time series)
5. **Use sophisticated imputation** (advanced techniques)

In [ ]:
# Let's handle missing values strategically

# For coffee consumption: fill with median (some students might not drink coffee: coffee_cups_per_day)
# Hint: Use `df['column'].median()` to get the median value.
coffee_median = ???
df['coffee_cups_per_day'] = df['coffee_cups_per_day'].fillna(coffee_median)
print(f"☕ Filled missing coffee values with median: {coffee_median:.2f}")

# For stress level: fill with mean (stress is continuous: stress_level)
# Hint: Same approach as above, but use mean instead of median.
stress_mean = ???
df['stress_level'] = df['stress_level'].fillna(stress_mean)
print(f"😰 Filled missing stress values with mean: {stress_mean:.2f}")

# For funding: fill with department-specific means
# Hint: Group by 'department_clean', calculate the mean of 'funding_amount'.
funding_by_dept = df.groupby('department_clean')['funding_amount'].mean()
# Now fill missing funding amounts with the mean for that department using `map`.
df['funding_amount'] = df['funding_amount'].fillna(
    df['department_clean'].map(funding_by_dept)
)
print(f"💰 Filled missing funding values with department-specific means")

# Check our work
print(f"\n✅ Missing values after cleaning: {df.isnull().sum().sum()}")

## Step 2: Remove Duplicates

Duplicate rows can skew our analysis:

In [ ]:
# Check for duplicates
print(f"🔄 Duplicate rows: {df.duplicated().sum()}")

# Look at some duplicates
if df.duplicated().sum() > 0:
    print("\n👀 Some duplicate rows:")
    duplicate_rows = df[df.duplicated(keep=False)].sort_values('student_id') # Show all duplicates
    print(duplicate_rows[['student_id', 'department_clean', 'years_in_program', 'papers_published']].head(10))
    
    # Remove duplicates
    df.drop_duplicates(inplace=True) # Keep only the first occurrence
    print(f"\n✅ Removed duplicates. New shape: {df.shape}")

## Step 3: Handle Outliers

Let's find and deal with extreme values:

In [ ]:
# Check for coffee outliers
# Using quantile() to find Q1 and Q3
coffee_q75 = df['coffee_cups_per_day'].quantile(0.75)
coffee_q25 = df['coffee_cups_per_day'].quantile(0.25)
coffee_iqr = coffee_q75 - coffee_q25

# Define outliers as values beyond 1.5 * IQR from Q1 or Q3
coffee_lower_bound = coffee_q25 - 1.5 * coffee_iqr
coffee_upper_bound = coffee_q75 + 1.5 * coffee_iqr

coffee_outliers = df[(df['coffee_cups_per_day'] < coffee_lower_bound) | # "|" is logical OR 
                    (df['coffee_cups_per_day'] > coffee_upper_bound)]

print(f"☕ Coffee consumption outliers: {len(coffee_outliers)}")
print(f"   Normal range: {coffee_lower_bound:.2f} - {coffee_upper_bound:.2f} cups/day") 
print(f"   Outliers range: {coffee_outliers['coffee_cups_per_day'].min():.2f} - {coffee_outliers['coffee_cups_per_day'].max():.2f} cups/day")

# Let's be reasonable and cap extreme values
# Anyone drinking more than 10 cups/day might have data entry errors
extreme_coffee = df['coffee_cups_per_day'] > 10
print(f"\n🚨 Students with >10 cups/day: {extreme_coffee.sum()}")

# Cap at 8 cups/day (still high but reasonable for a stressed PhD student)
df.loc[df['coffee_cups_per_day'] > 8, 'coffee_cups_per_day'] = 8 # .loc is used for label-based indexing, while .iloc is for position-based indexing
print(f"✅ Capped extreme coffee consumption at 8 cups/day")

## Step 4: Feature Engineering

Let's create some new features that might be useful for our analysis:

In [ ]:
# Create a productivity score
df['productivity_score'] = (df['papers_published'] * 2 + df['conferences_attended']) / df['years_in_program']

# Create a work-life balance indicator
# High Hunt Library hours + high stress = poor work-life balance
df['work_life_balance'] = 10 - (df['hours_in_hunt_library_per_week'] / 10 + df['stress_level']) / 2
df['work_life_balance'] = df['work_life_balance'].clip(1, 10)  # Using clip() to keep it between 1-10

# Create a seniority category
# Using cut() to bin years_in_program into categories
df['seniority'] = pd.cut(df['years_in_program'], 
                        bins=[0, 2, 4, 10], 
                        labels=['Early', 'Mid', 'Advanced'])

# Create a caffeine dependency category
df['caffeine_level'] = pd.cut(df['coffee_cups_per_day'], 
                             bins=[0, 2, 4, 8], 
                             labels=['Low', 'Moderate', 'High'])

print("🛠️ New features created:")
print(f"   - productivity_score: {df['productivity_score'].describe()}")
print(f"   - work_life_balance: {df['work_life_balance'].describe()}")
print(f"   - seniority: {df['seniority'].value_counts()}")
print(f"   - caffeine_level: {df['caffeine_level'].value_counts()}")

## 🎯 Final Cleaning Exercise

Your turn! Let's finish cleaning our dataset:

In [ ]:
# TODO: Complete these final cleaning steps:
# 1. Replace the old 'department' column with 'department_clean'
# 2. Round numerical columns to reasonable decimal places
# 3. Reset the index after all our filtering
# 4. Create a final summary of our cleaned dataset

# Your code here:

# 1. Replace department column
df['department'] = ???
df.drop('department_clean', axis=1, inplace=True)

# 2. Round numerical columns
df['coffee_cups_per_day'] = df['coffee_cups_per_day'].round(2) # Coffee rounded to 2 decimal places
df['hours_in_hunt_library_per_week'] = df['hours_in_hunt_library_per_week'].round(1) # Hunt hours rounded to 1 decimal place
df['stress_level'] = df['stress_level'].round(2)
df['funding_amount'] = df['funding_amount'].round(2)
df['distance_from_campus_miles'] = df['distance_from_campus_miles'].round(2)
df['productivity_score'] = df['productivity_score'].round(3) # More precision for productivity

# Round work-life balance to 2 decimal places
df['work_life_balance'] = ???

# 3. Reset index after filtering (the indices may be non-sequential now)
df.reset_index(drop=True, inplace=True)

# 4. Final summary
print("🎉 Final cleaned dataset summary:")
print(f"   Shape: {df.shape}")
print(f"   Missing values: {df.isnull().sum().sum()}")
print(f"   Duplicates: {df.duplicated().sum()}")
print(f"   Columns: {len(df.columns)}")

print("\n📊 Final dataset preview:")
print(df.head())

## Save Our Clean Dataset

Let's save our cleaned dataset for use in Block 3:

In [ ]:
# Save to CSV
df.to_csv('phd_research_productivity_clean.csv', index=False)
print("💾 Clean dataset saved as 'phd_research_productivity_clean.csv'")

# Quick verification
print(f"\n✅ Verification - file size: {len(df)} rows, {len(df.columns)} columns")
print(f"✅ Ready for Block 3: Linear Regression Analysis!")

---

# 🎯 Bonus Section: Advanced Operations

For those who finish early or want to dive deeper!

## GroupBy Operations

One of Pandas' most powerful features:

In [ ]:
# Group by department and analyze
dept_analysis = df.groupby('department').agg({
    'papers_published': ['mean', 'std', 'max'],
    'coffee_cups_per_day': 'mean',
    'stress_level': 'mean',
    'funding_amount': 'mean',
    'productivity_score': 'mean'
})

print("🏢 Department Analysis:")
print(dept_analysis.round(2))

## Correlation Analysis

Let's see what factors are related to research productivity:

In [ ]:
# Select numerical columns for correlation
numerical_cols = ['years_in_program', 'papers_published', 'conferences_attended',
                  'coffee_cups_per_day', 'hours_in_hunt_library_per_week',
                  'advisor_meetings_per_month', 'stress_level', 'funding_amount',
                  'distance_from_campus_miles', 'productivity_score', 'work_life_balance']

correlation_matrix = df[numerical_cols].corr()

# Focus on correlations with papers_published
papers_correlations = correlation_matrix['papers_published'].sort_values(ascending=False)

print("📊 Correlations with Papers Published:")
for variable, correlation in papers_correlations.items():
    if variable != 'papers_published':
        print(f"   {variable}: {correlation:.3f}")

## Data Visualization Preview

A quick visualization to understand our data better:

In [ ]:
# Create a simple visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Papers by department
df.groupby('department')['papers_published'].mean().plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Average Papers by Department')
axes[0,0].set_ylabel('Papers Published')

# Coffee vs Papers scatter plot
axes[0,1].scatter(df['coffee_cups_per_day'], df['papers_published'], alpha=0.6)
axes[0,1].set_xlabel('Coffee Cups per Day')
axes[0,1].set_ylabel('Papers Published')
axes[0,1].set_title('Coffee vs Research Output')

# Stress level distribution
df['stress_level'].hist(bins=20, ax=axes[1,0])
axes[1,0].set_title('Stress Level Distribution')
axes[1,0].set_xlabel('Stress Level')
axes[1,0].set_ylabel('Frequency')

# Years vs Papers
axes[1,1].scatter(df['years_in_program'], df['papers_published'], alpha=0.6)
axes[1,1].set_xlabel('Years in Program')
axes[1,1].set_ylabel('Papers Published')
axes[1,1].set_title('Experience vs Research Output')

plt.tight_layout()
plt.show()

print("📈 Quick insights:")
print(f"   - Average papers published: {df['papers_published'].mean():.2f}")
print(f"   - Most productive department: {df.groupby('department')['papers_published'].mean().idxmax()}")
print(f"   - Coffee-paper correlation: {df['coffee_cups_per_day'].corr(df['papers_published']):.3f}")

## 🎯 Optional Exercise: Create Your Own Analysis

Now it's your turn to explore! Pick one of these questions and write the code to answer it:

In [ ]:
# Choose one of these questions and write code to answer it:

# Question 1: Do students who live farther from campus have better work-life balance?
# Question 2: Which combination of caffeine level and seniority is most productive?
# Question 3: Is there an optimal number of advisor meetings per month for productivity?
# Question 4: Create a "PhD Success Score" combining multiple factors

# Your exploratory code here:

# Example: Question 1 analysis
print("🏠 Distance vs Work-Life Balance Analysis:")

# Create distance categories
df['distance_category'] = pd.cut(df['distance_from_campus_miles'],
                                    bins=[0, 2, 5, 50],
                                    labels=['Close (<2 miles)', 'Medium (2-5 miles)', 'Far (>5 miles)'])

# Analyze work-life balance by distance
# Explicitly set observed=False to retain current behavior and silence the FutureWarning
distance_analysis = df.groupby('distance_category', observed=False)['work_life_balance'].agg(['mean', 'std', 'count'])
print(distance_analysis)

# Statistical test
close_wlb = df[df['distance_category'] == 'Close (<2 miles)']['work_life_balance']
far_wlb = df[df['distance_category'] == 'Far (>5 miles)']['work_life_balance']

print(f"\n📊 Students living close to campus: {close_wlb.mean():.2f} avg work-life balance")
print(f"📊 Students living far from campus: {far_wlb.mean():.2f} avg work-life balance")
print(f"📊 Difference: {far_wlb.mean() - close_wlb.mean():.2f}")

---

# 🎉 Congratulations!

## What We've Accomplished

In this block, you've learned to:

✅ **NumPy Fundamentals**
- Create and manipulate arrays
- Perform vectorized operations
- Calculate statistics efficiently
- Understand why NumPy is faster than pure Python

✅ **Pandas Mastery**
- Load and explore datasets
- Select, filter, and manipulate data
- Handle missing values strategically
- Remove duplicates and outliers
- Create new features from existing data

✅ **Real-World Data Cleaning**
- Standardized inconsistent categorical data
- Applied different missing value strategies
- Created meaningful derived features
- Prepared data for machine learning

## Your Clean Dataset

Our messy PhD student dataset is now clean and ready for analysis! In Block 3, we'll use this data to:
- Build a linear regression model
- Predict research productivity
- Interpret model results
- Understand what factors contribute to PhD success

## Key Takeaways

1. **Real data is messy** - cleaning is often 80% of the work
2. **NumPy and Pandas are complementary** - NumPy for computation, Pandas for data manipulation
3. **Always explore your data first** - understand before you clean
4. **Feature engineering matters** - new features can reveal hidden insights
5. **Document your cleaning decisions** - others (including future you) need to understand your choices

## Next Up: Block 3 🚀

In **Block 3: From Data to Insights**, we'll take our clean dataset and:
- Build predictive models with Scikit-learn
- Understand linear regression concepts
- Interpret model coefficients
- Evaluate model performance
- Make predictions about PhD student success!

Great work, future data scientists! 🎓✨